# SỬ DỤNG LẠI MÔ HÌNH ĐÃ ĐƯỢC HUẤN LUYỆN CỦA BÀI BÀO GỐC

In [2]:
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [3]:
import os

os.chdir(r"D:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\_original\Yeast")

label = pd.read_csv("label.csv")
feat_a = pd.read_csv("total_features_a.csv")
feat_b = pd.read_csv("total_features_b.csv")
feat_all = pd.concat([feat_a, feat_b], axis=1)
print(feat_all.shape)

(11188, 1232)


In [4]:
from tensorflow.keras.models import load_model
enc_a = load_model("AE_Yeast_a_.h5")
print(enc_a.summary())

X_no_enc = feat_all.values
y_no_enc = label.values

feat_a_enc = enc_a.predict(feat_a)
print(feat_a_enc.shape)

enc_b = load_model("AE_Yeast_b_.h5")
print(enc_b.summary())

feat_b_enc = enc_b.predict(feat_b)
print(feat_b_enc.shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 202)               124634    
_________________________________________________________________
dropout_2 (Dropout)          (None, 202)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 616)               125048    
Total params: 249,682
Trainable params: 249,682
Non-trainable params: 0
_________________________________________________________________
None
(11188, 616)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 202)               124634    
_________________________________________________________________
dropout_3 (Dropout)          (None, 202)               0         
________

In [5]:
import numpy as np

feat_all_enc = np.concatenate((feat_a_enc, feat_b_enc), axis=1)
print('Human new trainning set', feat_all_enc.shape)

Human new trainning set (11188, 1232)


In [6]:
import numpy as np

X_tr = np.copy(feat_all_enc)
y_tr = np.copy(label)
y_tr[y_tr == -1] = 0
print(y_tr)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [10]:
import lightgbm as lgb

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

gbm = lgb.LGBMClassifier(
    n_estimators=500, # 500
    num_leaves=50,  # 80
    learning_rate=0.05
    # max_depth=10,
    # feature_fraction=0.9,
    # bagging_fraction=0.8,
    # bagging_freq=5,
)

# gbm = lgb.LGBMClassifier(**params)

# pipeline = Pipeline([('transformer', t), ('estimator', gbm)])
cv = StratifiedKFold(n_splits=5, random_state=48, shuffle=True)
print(f'\n{cv.__str__()}...')
scr = cross_val_score(estimator=gbm, X=X_tr, y=y_tr, cv=cv, n_jobs=2)


StratifiedKFold(n_splits=5, random_state=48, shuffle=True)...


In [11]:
print(scr)
print("mean {:0.4f}, std {:0.3f}".format(np.mean(scr), np.std(scr)))

[0.95263628 0.95487042 0.95040214 0.9606616  0.95529727]
mean 0.9548, std 0.003


In [12]:
import lightgbm as lgb

gbm = lgb.LGBMClassifier(
    n_estimators=500,
    num_leaves=50,
    learning_rate=0.05
)

gbm.fit(X_tr, y_tr.ravel())

LGBMClassifier(learning_rate=0.05, n_estimators=500, num_leaves=50)

# TEST TRÊN TẬP ĐỘC LẬP

In [13]:
import os
os.chdir(r"D:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\Independent Species\Ecoli")
feat_a = pd.read_csv("total_features_a.csv")
feat_a = enc_a.predict(feat_a)

feat_b = pd.read_csv("total_features_b.csv")
feat_b = enc_b.predict(feat_b)

data_te = np.concatenate((feat_a, feat_b), axis=1)
print('data_te', data_te.shape)
label_te = [1] * data_te.shape[0]

data_te (6954, 1232)


In [14]:
print(gbm.score(data_te, label_te))
print("{:.4f}".format(gbm.score(data_te, label_te)))

0.8771929824561403
0.8772
